# 7. Extracting Information from Text

학습 목표
1. 비구조화된 데이터에서 구조화된(예, 표 등) 데이터을 추출할 수 있는 시스템 개발
2. 텍스트에서 개체와 관계를 밝혀낼 수 있는 로버스트한 방법은 무엇인가
3. 이러한 일에 적합한 corpora는 무엇인지, 어떻게 학습하고 평가하는데 사용할 수 있는가

## 1. Information Extraction

Structured data: regular, predictable organization of entities and relationships

ex) company와 location

![](image/table_1.1.png)

table 1.1의 데이터가 python에 tuple로 저장되어 있으면 (entity, relation, entity)

**"Which organizations operate in Atlanta?"** 이 질문은 아래의 코드로 표현할 수 있다.

In [3]:
locs = [('Omnicom', 'IN', 'New York'),
('DDB Needham', 'IN', 'New York'),
('Kaplan Thaler Group', 'IN', 'New York'),
('BBDO South', 'IN', 'Atlanta'),
('Georgia-Pacific', 'IN', 'Atlanta')]
query = [e1 for (e1, rel, e2) in locs if e2=='Atlanta']
print(query)

['BBDO South', 'Georgia-Pacific']


![](image/table_1.2.png)

(1)을 읽어보면 Atlanta에 위치한 회사를 알아낼 수 있지만 컴퓨터는 어떻게 알아내도록 할 수 있을까?

그리고 (1)은 table 1.1과 다르게 unstructured data

Information Extraction: getting meaning from text

### 1.1   Information Extraction Architecture

raw text -> sentence -> words -> POS tagging -> entity detection -> relation detection

![](image/F1.1.PNG)

In [4]:
import nltk, re, pprint

In [5]:
def ie_preprocess(document):
    sentences = nltk.sent_tokenize(document) # sentence segmenter
    sentences = [nltk.word_tokenize(sent) for sent in sentences] # word tokenizer
    sentences = [nltk.pos_tag(sent) for sent in sentences] # pos tagger

entity detection:
Segment and lable the entities

relation detection:
search for specific patterns between pairs of entities that occur near on another in the text



## 2. Chunking

entity detection의 basic techinique
- chunking : segments and labels multi-token sequences 
- 작은 박스는 part-of-speech tagging
- 큰 박스는 higher-level chunking
- tokenize 처럼 빈칸 제외

![](image/figure_2.1.png)

### 2.1   Noun Phrase Chunking

NP-chunking: noun phrase에 해당하는 chunk 찾기

the market for system-management software for Digital's hardware (single noun phrase, two nested noun phrase)

NP-chunks capture only the market

NP-chunk가 다른 NP-chunk를 포함하지 않도록 정의되어 있어서

prepositional phrase(전치사구)나 subordinate clause(종속절)은 NP-chunk에 포함되지 않음

이미 이 noun phrase에 noun phrase가 더 포함되어 있기 때문에

NP-chunking의 유용한 소스는 Part-of-speech tags

chunk grammar
- 문장이 어떻게 chunk로 나뉘어야하는지 규칙을 명시

In [6]:
sentence = [("the", "DT"), ("little", "JJ"), ("yellow", "JJ"),
("dog", "NN"), ("barked", "VBD"), ("at", "IN"),  ("the", "DT"), ("cat", "NN")]
grammar = "NP: {<DT>?<JJ>*<NN>}" # single regular-expression rule
cp = nltk.RegexpParser(grammar) # chunk parser 
result = cp.parse(sentence) # test
print(result)

(S
  (NP the/DT little/JJ yellow/JJ dog/NN)
  barked/VBD
  at/IN
  (NP the/DT cat/NN))


In [15]:
result.draw()

![](image/2.2_result.png)

### 2.2 Tag Patterns

In [12]:
sentence = [("the", "DT"), ("little", "JJ"), ("yellow", "JJ"),
("dog", "NN"), ("barked", "VBD"), ("at", "IN"),  ("the", "DT"), ("cat", "NN")]
sentence

[('the', 'DT'),
 ('little', 'JJ'),
 ('yellow', 'JJ'),
 ('dog', 'NN'),
 ('barked', 'VBD'),
 ('at', 'IN'),
 ('the', 'DT'),
 ('cat', 'NN')]

In [13]:
grammar = "NP: {<DT>?<JJ.*>*<NN.*>+.}"
cp = nltk.RegexpParser(grammar)
result = cp.parse(sentence)
print(result)

(S the/DT little/JJ yellow/JJ dog/NN barked/VBD at/IN the/DT cat/NN)


### 2.3   Chunking with Regular Expressions

In [29]:
# grammar = r"""
#   NP: {<DT|PP\$>?<JJ>*<NN>}   # chunk determiner/possessive, adjectives and noun
#       {<NNP>+}                # chunk sequences of proper nouns
# """
grammar = r"NP: {<DT|PP\$>?<JJ>*<NN>}"
cp = nltk.RegexpParser(grammar)
sentence = [("Rapunzel", "NNP"), ("let", "VBD"), ("down", "RP"), 
                 ("her", "PP$"), ("long", "JJ"), ("golden", "JJ"), ("hair", "NN")]
print(cp.parse(sentence))

(S
  Rapunzel/NNP
  let/VBD
  down/RP
  (NP her/PP$ long/JJ golden/JJ hair/NN))


In [14]:
nouns = [("money", "NN"), ("market", "NN"), ("fund", "NN")]
grammar = "NP: {<NN><NN>}  # Chunk two consecutive nouns"
cp = nltk.RegexpParser(grammar)
print(cp.parse(nouns))

(S (NP money/NN market/NN) fund/NN)


In [7]:
nouns = [("money", "NN"), ("market", "NN"), ("fund", "NN")]
grammar = "NP: {<NN>+}  # Chunk two consecutive nouns"
cp = nltk.RegexpParser(grammar)
print(cp.parse(nouns))

(S (NP money/NN market/NN fund/NN))


### 2.4   Exploring Text Corpora

In [15]:
cp = nltk.RegexpParser('CHUNK: {<V.*> <TO> <V.*>}')
brown = nltk.corpus.brown
for sent in brown.tagged_sents():
    tree = cp.parse(sent)
    for subtree in tree.subtrees():
        if subtree.label() == 'CHUNK': print(subtree)


(CHUNK combined/VBN to/TO achieve/VB)
(CHUNK continue/VB to/TO place/VB)
(CHUNK serve/VB to/TO protect/VB)
(CHUNK wanted/VBD to/TO wait/VB)
(CHUNK allowed/VBN to/TO place/VB)
(CHUNK expected/VBN to/TO become/VB)
(CHUNK expected/VBN to/TO approve/VB)
(CHUNK expected/VBN to/TO make/VB)
(CHUNK intends/VBZ to/TO make/VB)
(CHUNK seek/VB to/TO set/VB)
(CHUNK like/VB to/TO see/VB)
(CHUNK designed/VBN to/TO provide/VB)
(CHUNK get/VB to/TO hear/VB)
(CHUNK expects/VBZ to/TO tell/VB)
(CHUNK expected/VBN to/TO give/VB)
(CHUNK prefer/VB to/TO pay/VB)
(CHUNK required/VBN to/TO obtain/VB)
(CHUNK permitted/VBN to/TO teach/VB)
(CHUNK designed/VBN to/TO reduce/VB)
(CHUNK Asked/VBN to/TO elaborate/VB)
(CHUNK got/VBN to/TO go/VB)
(CHUNK raised/VBN to/TO pay/VB)
(CHUNK scheduled/VBN to/TO go/VB)
(CHUNK cut/VBN to/TO meet/VB)
(CHUNK needed/VBN to/TO meet/VB)
(CHUNK hastened/VBD to/TO add/VB)
(CHUNK found/VBN to/TO prevent/VB)
(CHUNK continue/VB to/TO insist/VB)
(CHUNK compelled/VBN to/TO make/VB)
(CHUNK mad

### 2.5   Chinking

chunk에서 어떤걸 뺄지 결정하는게 더 쉽다
chink는 chunk에 포함되지 않을 token sequence를 의미함

![](image/table_2.1.png)

In [8]:
grammar = r"""
  NP:
    {<.*>+}          # Chunk everything
    }<VBD|IN>+{      # Chink sequences of VBD and IN
  """
sentence = [("the", "DT"), ("little", "JJ"), ("yellow", "JJ"),
       ("dog", "NN"), ("barked", "VBD"), ("at", "IN"),  ("the", "DT"), ("cat", "NN")]
cp = nltk.RegexpParser(grammar)
sentence

[('the', 'DT'),
 ('little', 'JJ'),
 ('yellow', 'JJ'),
 ('dog', 'NN'),
 ('barked', 'VBD'),
 ('at', 'IN'),
 ('the', 'DT'),
 ('cat', 'NN')]

In [9]:
print(cp.parse(sentence))

(S
  (NP the/DT little/JJ yellow/JJ dog/NN)
  barked/VBD
  at/IN
  (NP the/DT cat/NN))


### 2.6   Representing Chunks: Tags vs Trees

chunk는 tag 또는 tree로 표현할 수 있다.

많이 사용되는 표현법은 IOB tags. 

각 토큰은 I (inside), O (outside), or B (begin) 중에 하나로 태그됨

![](image/figure_2.5.png)

![](image/figure_2.6.png)

## 3   Developing and Evaluating Chunkers

In [ ]:
how to evaluate chunkers

how to score the accuracy of a chunker relative to a corpus

### 3.1   Reading IOB Format and the CoNLL 2000 Corpus

Wall Street Journal text that has been tagged then chunked using the IOB notation.

In [18]:
text = '''
he PRP B-NP
accepted VBD B-VP
the DT B-NP
position NN I-NP
of IN B-PP
vice NN B-NP
chairman NN I-NP
of IN B-PP
Carlyle NNP B-NP
Group NNP I-NP
, , O
a DT B-NP
merchant NN I-NP
banking NN I-NP
concern NN I-NP
. . O
'''
nltk.chunk.conllstr2tree(text, chunk_types=['NP']).draw()

![](image/result_3.1.png)

In [21]:
from nltk.corpus import conll2000
print(conll2000.chunked_sents('train.txt')[99])

(S
  (PP Over/IN)
  (NP a/DT cup/NN)
  (PP of/IN)
  (NP coffee/NN)
  ,/,
  (NP Mr./NNP Stone/NNP)
  (VP told/VBD)
  (NP his/PRP$ story/NN)
  ./.)


In [22]:
print(conll2000.chunked_sents('train.txt', chunk_types=['NP'])[99])

(S
  Over/IN
  (NP a/DT cup/NN)
  of/IN
  (NP coffee/NN)
  ,/,
  (NP Mr./NNP Stone/NNP)
  told/VBD
  (NP his/PRP$ story/NN)
  ./.)


### 3.2   Simple Evaluation and Baselines

In [30]:
from nltk.corpus import conll2000
cp = nltk.RegexpParser("")
test_sents = conll2000.chunked_sents('test.txt', chunk_types=['NP'])
print(cp.evaluate(test_sents))

ChunkParse score:
    IOB Accuracy:  43.4%%
    Precision:      0.0%%
    Recall:         0.0%%
    F-Measure:      0.0%%


In [32]:
test_sents

[Tree('S', [Tree('NP', [('Rockwell', 'NNP'), ('International', 'NNP'), ('Corp.', 'NNP')]), Tree('NP', [("'s", 'POS'), ('Tulsa', 'NNP'), ('unit', 'NN')]), ('said', 'VBD'), Tree('NP', [('it', 'PRP')]), ('signed', 'VBD'), Tree('NP', [('a', 'DT'), ('tentative', 'JJ'), ('agreement', 'NN')]), ('extending', 'VBG'), Tree('NP', [('its', 'PRP$'), ('contract', 'NN')]), ('with', 'IN'), Tree('NP', [('Boeing', 'NNP'), ('Co.', 'NNP')]), ('to', 'TO'), ('provide', 'VB'), Tree('NP', [('structural', 'JJ'), ('parts', 'NNS')]), ('for', 'IN'), Tree('NP', [('Boeing', 'NNP')]), Tree('NP', [("'s", 'POS'), ('747', 'CD'), ('jetliners', 'NNS')]), ('.', '.')]), Tree('S', [Tree('NP', [('Rockwell', 'NNP')]), ('said', 'VBD'), Tree('NP', [('the', 'DT'), ('agreement', 'NN')]), ('calls', 'VBZ'), ('for', 'IN'), Tree('NP', [('it', 'PRP')]), ('to', 'TO'), ('supply', 'VB'), Tree('NP', [('200', 'CD'), ('additional', 'JJ'), ('so-called', 'JJ'), ('shipsets', 'NNS')]), ('for', 'IN'), Tree('NP', [('the', 'DT'), ('planes', 'NNS')

In [31]:
cp.parse(test_sents)

[Tree('S', [Tree('NP', [('Rockwell', 'NNP'), ('International', 'NNP'), ('Corp.', 'NNP')]), Tree('NP', [("'s", 'POS'), ('Tulsa', 'NNP'), ('unit', 'NN')]), ('said', 'VBD'), Tree('NP', [('it', 'PRP')]), ('signed', 'VBD'), Tree('NP', [('a', 'DT'), ('tentative', 'JJ'), ('agreement', 'NN')]), ('extending', 'VBG'), Tree('NP', [('its', 'PRP$'), ('contract', 'NN')]), ('with', 'IN'), Tree('NP', [('Boeing', 'NNP'), ('Co.', 'NNP')]), ('to', 'TO'), ('provide', 'VB'), Tree('NP', [('structural', 'JJ'), ('parts', 'NNS')]), ('for', 'IN'), Tree('NP', [('Boeing', 'NNP')]), Tree('NP', [("'s", 'POS'), ('747', 'CD'), ('jetliners', 'NNS')]), ('.', '.')]), Tree('S', [Tree('NP', [('Rockwell', 'NNP')]), ('said', 'VBD'), Tree('NP', [('the', 'DT'), ('agreement', 'NN')]), ('calls', 'VBZ'), ('for', 'IN'), Tree('NP', [('it', 'PRP')]), ('to', 'TO'), ('supply', 'VB'), Tree('NP', [('200', 'CD'), ('additional', 'JJ'), ('so-called', 'JJ'), ('shipsets', 'NNS')]), ('for', 'IN'), Tree('NP', [('the', 'DT'), ('planes', 'NNS')

IOB 태그의 정확도는 단어 중 3 분의 1 이상이 O (즉, NP 청크가 아닌) 태그가 있음을 나타냅니다. 그러나, 우리의 tagger가 어떤 덩어리도 찾지 못했기 때문에, 그것의 정확성, 리콜, f-measure는 모두 0입니다. 

이제 명사구 태그 (예 : CD, DT 및 JJ)의 특징 인 문자로 시작하는 태그를 찾는 순진한 정규식 청크를 사용해 보겠습니다.

In [33]:
grammar = r"NP: {<[CDJNP].*>+}"
cp = nltk.RegexpParser(grammar)
print(cp.evaluate(test_sents))


ChunkParse score:
    IOB Accuracy:  87.7%%
    Precision:     70.6%%
    Recall:        67.8%%
    F-Measure:     69.2%%


보시다시피이 접근법은 적절한 결과를 얻습니다. 그러나 우리는 각 파트 별 태그에 대해 가장 가능성이 큰 청크 태그 (I, O 또는 B)를 찾기 위해 교육 자료를 사용하는보다 데이터 중심의 접근 방식을 채택함으로써 이를 개선 할 수 있습니다. 

즉 unigram tagger (4)를 사용하여 chunker를 만들 수 있습니다. 
그러나 각 단어의 올바른 품사 태그를 결정하기보다는 각 단어의 품사 태그가 주어지면 올바른 청크 태그를 결정하려고합니다.

In [34]:
cp.parse(test_sents)

KeyboardInterrupt: 

In [31]:
class UnigramChunker(nltk.ChunkParserI):
    def __init__(self, train_sents): 
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]
        self.tagger = nltk.UnigramTagger(train_data)

    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)

In [30]:
# tree2conlltags to map each chunk tree to a list of word,tag,chunk triples

In [11]:
from nltk.corpus import conll2000
test_sents = conll2000.chunked_sents('test.txt', chunk_types=['NP'])
train_sents = conll2000.chunked_sents('train.txt', chunk_types=['NP'])

In [32]:
unigram_chunker = UnigramChunker(train_sents)
print(unigram_chunker.evaluate(test_sents))

ChunkParse score:
    IOB Accuracy:  92.9%%
    Precision:     79.9%%
    Recall:        86.8%%
    F-Measure:     83.2%%


In [33]:
postags = sorted(set(pos for sent in train_sents 
                     for (word,pos) in sent.leaves()))
print(unigram_chunker.tagger.tag(postags))

[('#', 'B-NP'), ('$', 'B-NP'), ("''", 'O'), ('(', 'O'), (')', 'O'), (',', 'O'), ('.', 'O'), (':', 'O'), ('CC', 'O'), ('CD', 'I-NP'), ('DT', 'B-NP'), ('EX', 'B-NP'), ('FW', 'I-NP'), ('IN', 'O'), ('JJ', 'I-NP'), ('JJR', 'B-NP'), ('JJS', 'I-NP'), ('MD', 'O'), ('NN', 'I-NP'), ('NNP', 'I-NP'), ('NNPS', 'I-NP'), ('NNS', 'I-NP'), ('PDT', 'B-NP'), ('POS', 'B-NP'), ('PRP', 'B-NP'), ('PRP$', 'B-NP'), ('RB', 'O'), ('RBR', 'O'), ('RBS', 'B-NP'), ('RP', 'O'), ('SYM', 'O'), ('TO', 'O'), ('UH', 'O'), ('VB', 'O'), ('VBD', 'O'), ('VBG', 'O'), ('VBN', 'O'), ('VBP', 'O'), ('VBZ', 'O'), ('WDT', 'B-NP'), ('WP', 'B-NP'), ('WP$', 'B-NP'), ('WRB', 'O'), ('``', 'O')]


In [34]:
class BigramChunker(nltk.ChunkParserI):
    def __init__(self, train_sents): 
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]
        self.tagger = nltk.BigramTagger(train_data)

    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)

In [36]:
bigram_chunker = BigramChunker(train_sents)
print(bigram_chunker.evaluate(test_sents))

ChunkParse score:
    IOB Accuracy:  93.3%%
    Precision:     82.3%%
    Recall:        86.8%%
    F-Measure:     84.5%%


### 3.3   Training Classifier-Based Chunkers

In [ ]:
두 문장이 동일한 품사 태그를 가지고 있지만, 서로 다르게 chunk 된다.
farmer rice는 별도의 덩어리이지만, computer monitor는 하나의 덩어리
chunking 성능을 높이려면 품사 태그뿐만 아니라 단어 내용에 대한 정보를 활용해야함 

In [12]:
class ConsecutiveNPChunkTagger(nltk.TaggerI): #1

    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in train_sents:
            untagged_sent = nltk.tag.untag(tagged_sent)
            history = []
            for i, (word, tag) in enumerate(tagged_sent):
                featureset = npchunk_features(untagged_sent, i, history) #2
                train_set.append( (featureset, tag) )
                history.append(tag)
        self.classifier = nltk.MaxentClassifier.train( #3
            train_set, algorithm='megam', trace=0)

    def tag(self, sentence):
        history = []
        for i, word in enumerate(sentence):
            featureset = npchunk_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)

class ConsecutiveNPChunker(nltk.ChunkParserI): #4
    def __init__(self, train_sents):
        tagged_sents = [[((w,t),c) for (w,t,c) in
                         nltk.chunk.tree2conlltags(sent)]
                        for sent in train_sents]
        self.tagger = ConsecutiveNPChunkTagger(tagged_sents)

    def parse(self, sentence):
        tagged_sents = self.tagger.tag(sentence)
        conlltags = [(w,t,c) for ((w,t),c) in tagged_sents]
        return nltk.chunk.conlltags2tree(conlltags)

In [14]:
def npchunk_features(sentence, i, history):
    word, pos = sentence[i]
    return {"pos": pos}

In [15]:
chunker = ConsecutiveNPChunker(train_sents)
print(chunker.evaluate(test_sents))

[Found megam: /usr/local/bin/megam]
[Found megam: /usr/local/bin/megam.opt]


Exception ignored in: <generator object find_file_iter at 0x110d08410>
RuntimeError: generator ignored GeneratorExit


ChunkParse score:
    IOB Accuracy:  92.9%%
    Precision:     79.9%%
    Recall:        86.7%%
    F-Measure:     83.2%%


In [ ]:
이전 품사 태그에 대한 기능을 추가 할 수도 있습니다. 
이 기능을 추가하면 분류자가 인접 태그 간의 상호 작용을 모델링 할 수 있으며 
bigram 청크와 밀접하게 관련된 청크가 됩니다.

In [16]:
def npchunk_features(sentence, i, history):
    word, pos = sentence[i]
    if i == 0:
        prevword, prevpos = "<START>", "<START>"
    else:
        prevword, prevpos = sentence[i-1]
    return {"pos": pos, "prevpos": prevpos}
chunker = ConsecutiveNPChunker(train_sents)
print(chunker.evaluate(test_sents))

ChunkParse score:
    IOB Accuracy:  93.6%%
    Precision:     81.9%%
    Recall:        87.2%%
    F-Measure:     84.5%%


In [21]:
def npchunk_features(sentence, i, history):
    word, pos = sentence[i]
    if i == 0:
        prevword, prevpos = "<START>", "<START>"
    else:
        prevword, prevpos = sentence[i-1]
        return {"pos": pos, "word": word, "prevpos": prevpos}

In [22]:
chunker = ConsecutiveNPChunker(train_sents)
print(chunker.evaluate(test_sents))

AttributeError: 'NoneType' object has no attribute 'items'

In [ ]:
마지막으로 Lookahead 기능, 쌍으로 된 기능 및 복잡한 상황 별 기능과 같은 
다양한 추가 기능을 사용하여 기능 추출기를 확장 할 수 있습니다. 
tags-since-dt라고하는이 마지막 기능은 가장 최근의 결정자 이후에 발생한 
모든 품사 태그 세트를 설명하는 문자열을 생성합니다. 
또는 색인 i보다 먼저 결정자가없는 경우 문장의 시작 부분부터 . .

In [18]:
def npchunk_features(sentence, i, history):
    word, pos = sentence[i]
    if i == 0:
        prevword, prevpos = "<START>", "<START>"
    else:
        prevword, prevpos = sentence[i-1]
    if i == len(sentence)-1:
        nextword, nextpos = "<END>", "<END>"
    else:
        nextword, nextpos = sentence[i+1]
    return {"pos": pos,
            "word": word,
            "prevpos": prevpos,
            "nextpos": nextpos, 
            "prevpos+pos": "%s+%s" % (prevpos, pos),
            "pos+nextpos": "%s+%s" % (pos, nextpos),
            "tags-since-dt": tags_since_dt(sentence, i)}  

In [19]:
def tags_since_dt(sentence, i):
    tags = set()
    for word, pos in sentence[:i]:
        if pos == 'DT':
            tags = set()
        else:
            tags.add(pos)
    return '+'.join(sorted(tags))

In [20]:
chunker = ConsecutiveNPChunker(train_sents)
print(chunker.evaluate(test_sents))

ChunkParse score:
    IOB Accuracy:  95.9%%
    Precision:     88.5%%
    Recall:        90.9%%
    F-Measure:     89.7%%


## 4   Recursion in Linguistic Structure

### 4.1   Building Nested Structure with Cascaded Chunkers

지금까지 우리 청크 구조는 비교적 평평했습니다. 나무는 NP와 같은 청크 노드 아래에 선택적으로 그룹화 된 태그가있는 토큰으로 구성됩니다. 그러나 재귀 규칙을 포함하는 다단계 청크 문법을 작성하여 임의의 깊이의 청크 구조를 구축하는 것이 가능합니다. 4.1에는 명사구, 전치사구, 동사구 및 문장에 대한 패턴이 있습니다. 이것은 4 단계 청크 문법이며 최대 4 개의 깊이를 갖는 구조를 만드는 데 사용할 수 있습니다.

In [41]:
grammar = r"""
  NP: {<DT|JJ|NN.*>+}          # Chunk sequences of DT, JJ, NN
  PP: {<IN><NP>}               # Chunk prepositions followed by NP
  VP: {<VB.*><NP|PP|CLAUSE>+$} # Chunk verbs and their arguments
  CLAUSE: {<NP><VP>}           # Chunk NP, VP
  """
cp = nltk.RegexpParser(grammar)
sentence = [("Mary", "NN"), ("saw", "VBD"), ("the", "DT"), ("cat", "NN"),
    ("sit", "VB"), ("on", "IN"), ("the", "DT"), ("mat", "NN")]

In [42]:
print(cp.parse(sentence))

(S
  (NP Mary/NN)
  saw/VBD
  (CLAUSE
    (NP the/DT cat/NN)
    (VP sit/VB (PP on/IN (NP the/DT mat/NN)))))


In [43]:
sentence = [("John", "NNP"), ("thinks", "VBZ"), ("Mary", "NN"),
    ("saw", "VBD"), ("the", "DT"), ("cat", "NN"), ("sit", "VB"),
    ("on", "IN"), ("the", "DT"), ("mat", "NN")]
print(cp.parse(sentence))

(S
  (NP John/NNP)
  thinks/VBZ
  (NP Mary/NN)
  saw/VBD
  (CLAUSE
    (NP the/DT cat/NN)
    (VP sit/VB (PP on/IN (NP the/DT mat/NN)))))


In [44]:
cp = nltk.RegexpParser(grammar, loop=2)
print(cp.parse(sentence))

(S
  (NP John/NNP)
  thinks/VBZ
  (CLAUSE
    (NP Mary/NN)
    (VP
      saw/VBD
      (CLAUSE
        (NP the/DT cat/NN)
        (VP sit/VB (PP on/IN (NP the/DT mat/NN)))))))


### 4.2   Trees

![](image/result_2.png)

In [ ]:
S is the parent of VP; conversely VP is a child of S. 
Also, since NP and VP are both children of S, they are also siblings.

In [42]:
tree1 = nltk.Tree('NP', ['Alice'])
print(tree1)

(NP Alice)


In [43]:
tree2 = nltk.Tree('NP', ['the', 'rabbit'])
print(tree2)

(NP the rabbit)


In [44]:
tree3 = nltk.Tree('VP', ['chased', tree2])
tree4 = nltk.Tree('S', [tree1, tree3])
print(tree4)

(S (NP Alice) (VP chased (NP the rabbit)))


In [45]:
print(tree4[1])

(VP chased (NP the rabbit))


In [46]:
tree4[1].label()

'VP'

In [47]:
tree4.leaves()

['Alice', 'chased', 'the', 'rabbit']

In [48]:
tree4[1][1][1]

'rabbit'

In [49]:
tree3.draw() 

![](image/result_.png)

### 4.3   Tree Traversal

In [35]:
def traverse(t):
    try:
        t.label()
    except AttributeError:
        print(t, end=" ")
    else:
        # Now we know that t.node is defined
        print('(', t.label(), end=" ")
        for child in t:
            traverse(child)
        print(')', end=" ")

In [37]:
import nltk

In [39]:
t = nltk.Tree('(S (NP Alice) (VP chased (NP the rabbit)))')


TypeError: Tree: Expected a node value and child list 

## 5   Named Entity Recognition

![](image/table_5.1.png)

명명 된 개체 인식 (NER) 시스템의 목표는 명명 된 개체의 모든 텍스트 설명을 식별하는 것입니다. 
NE의 경계를 식별하고 그 유형을 식별하는 두 가지 하위 작업으로 나눌 수 있습니다. 

명명 된 엔티티 인식은 정보 추출에서 관계를 식별하기위한 서곡 인 경우가 많지만 다른 작업에도 기여할 수 있습니다. 

예를 들어, 질문 응답 (QA)에서 우리는 전체 페이지가 아닌 사용자의 질문에 대한 답변을 포함하는 부분을 복구하여 정보 검색의 정확성을 향상시킵니다. 

대부분의 QA 시스템은 표준 정보 검색에 의해 반환 된 문서를 취한 다음 해답이 포함 된 문서에서 최소 텍스트 스니펫을 분리하려고 시도합니다. 
이제 질문은 누가 미국의 초대 대통령 이었습니까? 그리고 검색된 문서 중 하나에는 다음 내용이 포함되어 있습니다.

![](image/figure_5.1.png)

지명 사전은 많은 국가에서 좋은 위치를 차지하고 있으며 도미니카 공화국의 산체스 (Sanchez)와 베트남의 국가 (On)와 같은 위치를 잘못 찾습니다. 물론 지명 사전에서 그러한 위치를 생략 할 수는 있지만 문서에 나타날 때이를 식별 할 수는 없습니다.

사람이나 조직의 이름이 더 어려워집니다. 그러한 이름의 목록에는 잘못된 적용 범위가있을 수 있습니다. 새로운 조직이 매일 생겨나므로 현대의 뉴스 또는 블로그 항목을 다루려고한다면 지명 사전을 사용하여 많은 엔티티를 인식 할 수 없을 것입니다.

어려움의 또 다른 주요 원인은 많은 명명 된 엔티티 용어가 모호하다는 사실 때문입니다. 따라서 5 월과 북한은 각각 DATE과 LOCATION의 명명 된 엔티티의 일부가 될 가능성이 있지만 둘 다 PERSON의 일부가 될 수 있습니다. 

반대로 Christian Dior는 PERSON처럼 보이지만 ORGANIZATION 유형이 될 가능성이 큽니다. 양키 (Yankee)와 같은 용어는 일부 컨텍스트에서는 일반 수정자가되지만 양키 내야 (the Yankee infielders)라는 구문에서는 ORGANIZATION 유형의 엔터티로 표시됩니다.

스탠포드 대학 (Stanford University)과 같은 여러 단어로 된 이름과 Cecil H. Green Library 및 Escondido Village Conference Service Center와 같은 다른 이름이 포함 된 이름이 추가로 챌린지를 제기합니다. 

따라서 명명 된 엔티티 인식에서는 다중 토큰 시퀀스의 시작과 끝을 식별 할 수 있어야합니다.

명명 된 엔티티 인식은 명사구 청킹 (chunking)에서 본 분류 기준 기반 접근 방식에 매우 적합한 작업입니다. 특히 IOB 형식을 사용하여 문장의 각 단어에 레이블을 붙이는 태그러를 만들 수 있습니다. 

여기서 청크는 적절한 유형으로 레이블이 지정됩니다. 다음은 CONLL 2002 (conll2002) 네덜란드어 훈련 데이터의 일부입니다.

In [53]:
sent = nltk.corpus.treebank.tagged_sents()[22]
print(nltk.ne_chunk(sent, binary=True))

(S
  The/DT
  (NE U.S./NNP)
  is/VBZ
  one/CD
  of/IN
  the/DT
  few/JJ
  industrialized/VBN
  nations/NNS
  that/WDT
  *T*-7/-NONE-
  does/VBZ
  n't/RB
  have/VB
  a/DT
  higher/JJR
  standard/NN
  of/IN
  regulation/NN
  for/IN
  the/DT
  smooth/JJ
  ,/,
  needle-like/JJ
  fibers/NNS
  such/JJ
  as/IN
  crocidolite/NN
  that/WDT
  *T*-1/-NONE-
  are/VBP
  classified/VBN
  *-5/-NONE-
  as/IN
  amphobiles/NNS
  ,/,
  according/VBG
  to/TO
  (NE Brooke/NNP)
  T./NNP
  Mossman/NNP
  ,/,
  a/DT
  professor/NN
  of/IN
  pathlogy/NN
  at/IN
  the/DT
  (NE University/NNP)
  of/IN
  (NE Vermont/NNP College/NNP)
  of/IN
  (NE Medicine/NNP)
  ./.)


In [54]:
print(nltk.ne_chunk(sent)) 

(S
  The/DT
  (GPE U.S./NNP)
  is/VBZ
  one/CD
  of/IN
  the/DT
  few/JJ
  industrialized/VBN
  nations/NNS
  that/WDT
  *T*-7/-NONE-
  does/VBZ
  n't/RB
  have/VB
  a/DT
  higher/JJR
  standard/NN
  of/IN
  regulation/NN
  for/IN
  the/DT
  smooth/JJ
  ,/,
  needle-like/JJ
  fibers/NNS
  such/JJ
  as/IN
  crocidolite/NN
  that/WDT
  *T*-1/-NONE-
  are/VBP
  classified/VBN
  *-5/-NONE-
  as/IN
  amphobiles/NNS
  ,/,
  according/VBG
  to/TO
  (PERSON Brooke/NNP T./NNP Mossman/NNP)
  ,/,
  a/DT
  professor/NN
  of/IN
  pathlogy/NN
  at/IN
  the/DT
  (ORGANIZATION University/NNP)
  of/IN
  (PERSON Vermont/NNP College/NNP)
  of/IN
  (GPE Medicine/NNP)
  ./.)


## 6   Relation Extraction

In [55]:
IN = re.compile(r'.*\bin\b(?!\b.+ing)')
for doc in nltk.corpus.ieer.parsed_docs('NYT_19980315'):
    for rel in nltk.sem.extract_rels('ORG', 'LOC', doc,
                                     corpus='ieer', pattern = IN):
        print(nltk.sem.rtuple(rel))

[ORG: 'WHYY'] 'in' [LOC: 'Philadelphia']
[ORG: 'McGlashan &AMP; Sarrail'] 'firm in' [LOC: 'San Mateo']
[ORG: 'Freedom Forum'] 'in' [LOC: 'Arlington']
[ORG: 'Brookings Institution'] ', the research group in' [LOC: 'Washington']
[ORG: 'Idealab'] ', a self-described business incubator based in' [LOC: 'Los Angeles']
[ORG: 'Open Text'] ', based in' [LOC: 'Waterloo']
[ORG: 'WGBH'] 'in' [LOC: 'Boston']
[ORG: 'Bastille Opera'] 'in' [LOC: 'Paris']
[ORG: 'Omnicom'] 'in' [LOC: 'New York']
[ORG: 'DDB Needham'] 'in' [LOC: 'New York']
[ORG: 'Kaplan Thaler Group'] 'in' [LOC: 'New York']
[ORG: 'BBDO South'] 'in' [LOC: 'Atlanta']
[ORG: 'Georgia-Pacific'] 'in' [LOC: 'Atlanta']


In [56]:
from nltk.corpus import conll2002
vnv = """
(
is/V|    # 3rd sing present and
was/V|   # past forms of the verb zijn ('be')
werd/V|  # and also present
wordt/V  # past of worden ('become)
)
.*       # followed by anything
van/Prep # followed by van ('of')
"""
VAN = re.compile(vnv, re.VERBOSE)
for doc in conll2002.chunked_sents('ned.train'):
    for r in nltk.sem.extract_rels('PER', 'ORG', doc,
                                   corpus='conll2002', pattern=VAN):
        print(nltk.sem.clause(r, relsym="VAN"))

VAN("cornet_d'elzius", 'buitenlandse_handel')
VAN('johan_rottiers', 'kardinaal_van_roey_instituut')
VAN('annie_lennox', 'eurythmics')


## 7   Summary

정보 추출 시스템은 특정 유형의 엔티티 및 관계에 대해 제한이 없는 텍스트의 큰 본문을 검색하고 
이를 사용하여 잘 조직 된 데이터베이스를 채웁니다. 
이러한 데이터베이스를 사용하여 특정 질문에 대한 답변을 찾을 수 있습니다.

정보 추출 시스템의 일반적인 아키텍처는 텍스트를 segmenting, tokenizing, and part-of-speech tagging 지정함으로써 시작됩니다. 
그런 다음 결과 데이터에 특정 유형의 엔티티가 검색됩니다. 
마지막으로, 정보 추출 시스템은 텍스트에서 서로 가까이에 언급 된 엔티티를 살펴보고 특정 엔티티간에 특정 관계가 유지되는지 여부를 확인하려고합니다.

엔티티 인식은 종종 다중 토큰 시퀀스를 세그먼트 화하고 적절한 엔티티 유형으로 레이블을 붙이는 청크를 사용하여 수행됩니다. 
일반적인 개체 유형에는 조직, 사람, 위치, 날짜, 시간, 돈 및 GPE (지역 정치 단체)가 포함됩니다.

Chunkers는 NLTK에서 제공하는 RegexpParser 클래스와 같은 규칙 기반 시스템을 사용하여 구성 할 수 있습니다. 
또는 이 장에서 제시 한 ConsecutiveNPChunker와 같은 기계 학습 기술을 사용하십시오. 
두 경우 모두 품사 태그는 대개 청크를 검색 할 때 매우 중요한 기능입니다.

chunkers는 비교적 평평한 데이터 구조를 생성하는 데 전문화되어 있지만 두 개의 덩어리가 겹치지 않는 곳에서는 함께 중첩되어 중첩 된 구조를 만들 수 있습니다.

관계 추출은 일반적으로 엔티티를 연결하는 텍스트의 특정 패턴과 중간 단어를 찾는 규칙 기반 시스템을 사용하여 수행 할 수 있습니다. 또는 일반적으로 훈련 코퍼스에서 자동으로 이러한 패턴을 학습하려고하는 기계 학습 시스템을 사용합니다.